In [ ]:
!git clone https://github.com/pratikkayal/PlantDoc-Object-Detection-Dataset.git

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as TF


In [ ]:
import os

def print_directory_sample(root_dir, indent=""):
    items = sorted(os.listdir(root_dir))
    dirs = [item for item in items if os.path.isdir(os.path.join(root_dir, item))]
    files = [item for item in items if os.path.isfile(os.path.join(root_dir, item))]
    
    # Print up to 10 files
    for fname in files[:10]:
        print(indent + "|-- " + fname)
    if len(files) > 10:
        print(indent + "    ...")
    
    # Recursively print directories
    for d in dirs:
        print(indent + "|-- " + d + "/")
        print_directory_sample(os.path.join(root_dir, d), indent + "    ")

# Set your dataset root, e.g., "."
root_path = "."
print_directory_sample(root_path)


In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn, FasterRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Dataset path setup
DATASET_PATH = "PlantDoc-Object-Detection-Dataset"
TRAIN_IMG_DIR = os.path.join(DATASET_PATH, "TRAIN")
TEST_IMG_DIR = os.path.join(DATASET_PATH, "TEST")
TRAIN_CSV = os.path.join(DATASET_PATH, "train_labels.csv")
TEST_CSV = os.path.join(DATASET_PATH, "test_labels.csv")

def load_annotations(csv_path):
    df = pd.read_csv(csv_path)
    return df[['filename', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax', 'class']]

train_df = load_annotations(TRAIN_CSV)
test_df = load_annotations(TEST_CSV)

class LeafDataset(Dataset):
    def __init__(self, df, img_folder, transforms=None):
        self.df = df
        self.img_folder = img_folder
        self.transforms = transforms
        self.imgs = df['filename'].unique()
        
        # Filter valid images
        self.valid_imgs = []
        print(f"Validating images in {img_folder}...")
        for img_name in self.imgs:
            img_path = os.path.join(img_folder, img_name)
            if os.path.exists(img_path):
                try:
                    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
                    if img is not None:
                        self.valid_imgs.append(img_name)
                    else:
                        print(f"Skipping corrupted: {img_name}")
                except:
                    print(f"Skipping unreadable: {img_name}")
            else:
                print(f"Skipping missing: {img_name}")
        
        self.imgs = self.valid_imgs
        print(f"Valid images: {len(self.imgs)}/{len(df['filename'].unique())}\n")

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        img_name = self.imgs[idx]
        img_path = os.path.join(self.img_folder, img_name)

        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if img is None:
            img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
            if img is None:
                raise ValueError(f"Image not found: {img_path}")

        # Convert to RGB
        if len(img.shape) == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        elif len(img.shape) == 3:
            if img.shape[2] == 1:
                img = cv2.cvtColor(img.squeeze(), cv2.COLOR_GRAY2RGB)
            elif img.shape[2] == 2:
                img = cv2.cvtColor(img[:, :, 0], cv2.COLOR_GRAY2RGB)
            elif img.shape[2] == 3:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            elif img.shape[2] == 4:
                img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

        img = np.ascontiguousarray(img, dtype=np.uint8)
        assert img.shape[2] == 3, f"Expected 3 channels, got {img.shape}"

        if self.transforms:
            img = self.transforms(img)
        
        assert img.shape[0] == 3, f"Expected 3 channels after transform, got {img.shape[0]}"

        boxes = []
        labels = []
        for _, row in self.df[self.df['filename'] == img_name].iterrows():
            boxes.append([row['xmin'], row['ymin'], row['xmax'], row['ymax']])
            labels.append(1)

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)
        target = {'boxes': boxes, 'labels': labels}

        return img, target

def get_transform():
    return T.Compose([T.ToTensor()])

def collate_fn(batch):
    return tuple(zip(*batch))

# Create datasets
train_dataset = LeafDataset(train_df, TRAIN_IMG_DIR, transforms=get_transform())
test_dataset = LeafDataset(test_df, TEST_IMG_DIR, transforms=get_transform())

# Create dataloaders
train_loader = DataLoader(
    train_dataset, 
    batch_size=4, 
    shuffle=True, 
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=1, 
    shuffle=False, 
    collate_fn=collate_fn
)

# Model setup
print("Setting up model...")
weights = FasterRCNN_ResNet50_FPN_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn(weights=weights)

# Replace box predictor
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=2)

# Device setup - SINGLE GPU ONLY (no DataParallel)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Using device: {device}")
if torch.cuda.device_count() > 1:
    print(f"Note: {torch.cuda.device_count()} GPUs available, but using single GPU")
    print("DataParallel is not compatible with Faster R-CNN in this setup")

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), 
    lr=5e-4
)



In [ ]:
# Training loop
num_epochs = 10
print(f"\nStarting training for {num_epochs} epochs...\n")

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    num_batches = 0
    
    for batch_idx, (images, targets) in enumerate(train_loader):
        # Move to device
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        # Forward pass
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        epoch_loss += losses.item()
        num_batches += 1
        
        # Print progress
        if batch_idx % 50 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {losses.item():.4f}")
    
    avg_loss = epoch_loss / num_batches if num_batches > 0 else 0
    print(f"\n{'='*60}")
    print(f"Epoch [{epoch + 1}/{num_epochs}] Complete - Average Loss: {avg_loss:.4f}")
    print(f"{'='*60}\n")

print("Training complete!")

# Save model
torch.save(model.state_dict(), 'leaf_disease_detector.pth')
print("Model saved as 'leaf_disease_detector.pth'")

In [ ]:
import os
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.transforms as T

# Load the trained model
def load_model(model_path, num_classes=2, device='cuda:0'):
    """Load the trained Faster R-CNN model"""
    model = fasterrcnn_resnet50_fpn(weights=None)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    # Load weights
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    
    print(f"Model loaded from {model_path}")
    return model

# Preprocess image
def preprocess_image(image_path):
    """Load and preprocess image for inference"""
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img is None:
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    
    if img is None:
        raise ValueError(f"Could not load image: {image_path}")
    
    # Convert to RGB
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif len(img.shape) == 3:
        if img.shape[2] == 1:
            img = cv2.cvtColor(img.squeeze(), cv2.COLOR_GRAY2RGB)
        elif img.shape[2] == 2:
            img = cv2.cvtColor(img[:, :, 0], cv2.COLOR_GRAY2RGB)
        elif img.shape[2] == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        elif img.shape[2] == 4:
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    
    img = np.ascontiguousarray(img, dtype=np.uint8)
    
    # Convert to tensor
    transform = T.Compose([T.ToTensor()])
    img_tensor = transform(img)
    
    return img, img_tensor

# Perform inference
def predict(model, image_tensor, device='cuda:0', confidence_threshold=0.75):
    """Run inference on an image"""
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        predictions = model([image_tensor])
    
    # Filter by confidence
    pred = predictions[0]
    keep = pred['scores'] > confidence_threshold
    
    boxes = pred['boxes'][keep].cpu().numpy()
    scores = pred['scores'][keep].cpu().numpy()
    labels = pred['labels'][keep].cpu().numpy()
    
    return boxes, scores, labels

# Visualize predictions
def visualize_predictions(image, boxes, scores, labels, title="Predictions", save_path=None):
    """Visualize bounding boxes on image"""
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)
    
    # Draw each bounding box
    for box, score, label in zip(boxes, scores, labels):
        xmin, ymin, xmax, ymax = box
        width = xmax - xmin
        height = ymax - ymin
        
        # Create rectangle
        rect = patches.Rectangle(
            (xmin, ymin), width, height,
            linewidth=2, edgecolor='red', facecolor='none'
        )
        ax.add_patch(rect)
        
        # Add label with confidence
        label_text = f'Leaf: {score:.2f}'
        ax.text(
            xmin, ymin - 5,
            label_text,
            color='white',
            fontsize=10,
            bbox=dict(facecolor='red', alpha=0.7, edgecolor='none', pad=2)
        )
    
    ax.set_title(f"{title} - {len(boxes)} detections", fontsize=14, fontweight='bold')
    ax.axis('off')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Saved visualization to {save_path}")
    
    plt.show()

# Test on a single image
def test_single_image(model, image_path, device='cuda:0', confidence_threshold=0.75, save_path=None):
    """Test model on a single image"""
    print(f"\nTesting on: {image_path}")
    
    # Load and preprocess
    image, image_tensor = preprocess_image(image_path)
    
    # Predict
    boxes, scores, labels = predict(model, image_tensor, device, confidence_threshold)
    
    print(f"Detected {len(boxes)} objects")
    for i, (box, score) in enumerate(zip(boxes, scores)):
        print(f"  Detection {i+1}: Confidence={score:.3f}, Box={box.astype(int)}")
    
    # Visualize
    visualize_predictions(
        image, boxes, scores, labels, 
        title=os.path.basename(image_path),
        save_path=save_path
    )
    
    return boxes, scores, labels

# Test on multiple images
def test_multiple_images(model, image_paths, device='cuda:0', confidence_threshold=0.5):
    """Test model on multiple images"""
    results = []
    
    for img_path in image_paths:
        try:
            boxes, scores, labels = test_single_image(
                model, img_path, device, confidence_threshold
            )
            results.append({
                'path': img_path,
                'boxes': boxes,
                'scores': scores,
                'labels': labels,
                'num_detections': len(boxes)
            })
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            results.append({
                'path': img_path,
                'error': str(e)
            })
    
    return results

# Test on test dataset
def test_on_test_set(model, test_loader, device='cuda:0', num_samples=5):
    """Test on images from test dataset"""
    model.eval()
    
    print(f"\nTesting on {num_samples} random test images...\n")
    
    for i, (images, targets) in enumerate(test_loader):
        if i >= num_samples:
            break
        
        image_tensor = images[0].to(device)
        target = targets[0]
        
        # Get predictions
        with torch.no_grad():
            predictions = model([image_tensor])
        
        pred = predictions[0]
        
        # Convert tensors to numpy
        image_np = image_tensor.cpu().permute(1, 2, 0).numpy()
        pred_boxes = pred['boxes'].cpu().numpy()
        pred_scores = pred['scores'].cpu().numpy()
        pred_labels = pred['labels'].cpu().numpy()
        gt_boxes = target['boxes'].cpu().numpy()
        
        # Filter predictions by confidence
        keep = pred_scores > 0.75
        pred_boxes = pred_boxes[keep]
        pred_scores = pred_scores[keep]
        
        # Visualize predictions vs ground truth
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Ground truth
        ax1.imshow(image_np)
        for box in gt_boxes:
            xmin, ymin, xmax, ymax = box
            rect = patches.Rectangle(
                (xmin, ymin), xmax-xmin, ymax-ymin,
                linewidth=2, edgecolor='green', facecolor='none'
            )
            ax1.add_patch(rect)
        ax1.set_title(f"Ground Truth ({len(gt_boxes)} boxes)", fontsize=12, fontweight='bold')
        ax1.axis('off')
        
        # Predictions
        ax2.imshow(image_np)
        for box, score in zip(pred_boxes, pred_scores):
            xmin, ymin, xmax, ymax = box
            rect = patches.Rectangle(
                (xmin, ymin), xmax-xmin, ymax-ymin,
                linewidth=2, edgecolor='red', facecolor='none'
            )
            ax2.add_patch(rect)
            ax2.text(
                xmin, ymin - 5, f'{score:.2f}',
                color='white', fontsize=9,
                bbox=dict(facecolor='red', alpha=0.7, edgecolor='none')
            )
        ax2.set_title(f"Predictions ({len(pred_boxes)} boxes)", fontsize=12, fontweight='bold')
        ax2.axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print(f"Image {i+1}: GT boxes={len(gt_boxes)}, Predicted boxes={len(pred_boxes)}\n")



In [ ]:

# ====================
# MAIN TESTING CODE
# ====================

# Load the trained model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = load_model('leaf_disease_detector.pth', num_classes=2, device=device)





In [ ]:
# Option 1: Test on specific image files


print("\n" + "="*60)
print("OPTION 1: Test on specific images")
print("="*60)

# Option 1: Test on specific image files
print("\n" + "="*60)
print("OPTION 1: Test on specific images")
print("="*60)

# Example: Test on images from TEST directory
test_images = [
    "/kaggle/input/field-like/test-samples/potato early blight/Pasted image (5).png"
]
# Filter to only existing images
test_images = [img for img in test_images if os.path.exists(img)]

if test_images:
    for img_path in test_images:
        test_single_image(model, img_path, device, confidence_threshold=0.75)
else:
    print("No test images found. Please provide valid image paths.")



In [ ]:
# Option 2: Test on test dataset loader
print("\n" + "="*60)
print("OPTION 2: Test on test dataset")
print("="*60)

# Load test dataset (assuming test_loader is already created)
try:
    test_on_test_set(model, test_loader, device, num_samples=5)
except NameError:
    print("test_loader not found. Please create it first using the training script.")

print("\nTesting complete!")

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

def test_single_image(model, image_path, device='cuda:0', confidence_threshold=0.75, save_path=None, crop_output_dir=None):
    """Test model on a single image, display boxes, and crop detections"""
    print(f"\nTesting on: {image_path}")
    
    # Load and preprocess
    image, image_tensor = preprocess_image(image_path)
    
    # Predict
    boxes, scores, labels = predict(model, image_tensor, device, confidence_threshold)
    
    print(f"Detected {len(boxes)} objects")
    for i, (box, score, label) in enumerate(zip(boxes, scores, labels)):
        print(f"  Detection {i+1}: Label={label}, Confidence={score:.3f}, Box={box.astype(int)}")
    
    # Draw bounding boxes on the original image
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    for box, score, label in zip(boxes, scores, labels):
        x1, y1, x2, y2 = box
        rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1,
                             fill=False, color='lime', linewidth=2)
        plt.gca().add_patch(rect)
        plt.text(x1, y1 - 5, f"{label} {score:.2f}",
                 color='yellow', fontsize=12, backgroundcolor="black")
    plt.axis("off")
    plt.title(os.path.basename(image_path))
    plt.show()

    # Crop and save/show each detection
    if crop_output_dir:
        os.makedirs(crop_output_dir, exist_ok=True)

    pil_img = Image.open(image_path).convert("RGB")
    cropped_images = []

    for i, (box, score, label) in enumerate(zip(boxes, scores, labels)):
        x1, y1, x2, y2 = map(int, box)
        cropped = pil_img.crop((x1, y1, x2, y2))
        cropped_images.append(cropped)

        # Show each cropped object
        plt.figure()
        plt.imshow(cropped)
        plt.title(f"{label} ({score:.2f})")
        plt.axis("off")
        plt.show()

        # Optionally save cropped images
        if crop_output_dir:
            crop_filename = f"{label}_{i+1}_{score:.2f}.png"
            crop_path = os.path.join(crop_output_dir, crop_filename)
            cropped.save(crop_path)
            print(f"Saved crop: {crop_path}")

    return boxes, scores, labels, cropped_images


# Example usage
test_images = [
    "/kaggle/input/field-like/test-samples/potato early blight/Pasted image (5).png"
]

# Filter to only existing images
test_images = [img for img in test_images if os.path.exists(img)]

if test_images:
    for img_path in test_images:
        test_single_image(
            model, img_path, device,
            confidence_threshold=0.75,
            crop_output_dir="cropped_detections"
        )
else:
    print("No test images found. Please provide valid image paths.")
